In [1]:
import pandas as pd
import numpy as np
import beam_search_module as bsm 
import ast
from plot_sg_and_pop import *
import json
from datetime import datetime, timedelta


In [2]:
def convert_timestamps(timestamp_str):
    # Remove the square brackets and split the string
    timestamps = timestamp_str.strip('[]').split(', ')
    # Extract the actual datetime strings and convert to datetime objects
    return [pd.Timestamp(ts.split('(')[-1].split(')')[0]) for ts in timestamps]


In [3]:
stock_df_all = pd.read_csv('data/stock_data_all_features.csv')
stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(ast.literal_eval)
stock_df_all['target'] = stock_df_all['target'].apply(ast.literal_eval)
stock_df_all['target'] = stock_df_all['target'].apply(lambda x: [((i / x[0])-1)*100 for i in x])
stock_df = stock_df_all.copy()
stock_df = stock_df.drop(['target', 'dates', 'symbol'], axis=1)
stock_df_backup = stock_df.copy()
stock_df_all


,symbol,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,...,3,8,3,26,34,16.82441,-17.72197,78.0,242.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.94, 3.71, -0...."
1,LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,...,1,1,1,2,5,18.59324,-8.00000,54.0,58.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 3.45, -2.0, -2...."
2,ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,...,2,9,3,28,37,11.23512,-6.57895,37.0,19.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -1.34, -1.36,..."
3,M07.F,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,...,4,4,2,28,15,9.97164,-10.13175,18.0,220.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -1.34, 0.0, 0.0..."
4,MUX.DE,Germany,Asset Management,EUR,Europe/Berlin,GER,Financial Services,11244.0,-2.942,3.008613e+08,...,0,5,2,29,19,14.24548,-26.26091,129.0,257.0,"[0.0, -2.56, -0.75, -0.38, 4.17, -1.64, 2.59, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7844,INRN.SW,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,...,4,4,2,15,13,17.62885,-12.85629,50.0,219.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -1.43, 2.49, 1..."
7845,RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,...,4,9,3,10,39,13.67960,-9.63956,245.0,54.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 2.97, 0.0, 0..."
7846,KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,...,4,9,3,26,39,9.76675,-8.39559,167.0,41.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0, 2.3..."
7847,MAW.L,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,...,4,9,3,26,39,14.62789,-10.74116,55.0,115.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [4]:

# Initialize start and end dates
start_date = datetime(2023, 10, 2)  # Monday, 2 October 2023
end_date = datetime(2024, 9, 29)    # Sunday, 29 September 2024

# Create a list of business days (Monday to Friday)
current_date = start_date
dates = []

# Loop through dates, adding only weekdays (Mon-Fri)
while current_date <= end_date:
    if current_date.weekday() < 5:  # 0 = Monday, 1 = Tuesday, ..., 4 = Friday
        dates.append(current_date)
    current_date += timedelta(days=1)

In [5]:
domain_attributes = True
time_series_attributes = True

if not domain_attributes and time_series_attributes:
    stock_df = stock_df.iloc[:, 11:]

if domain_attributes and not time_series_attributes:
    domain_features = stock_df.columns[:11]
    columns_keep = list(domain_features) + ['growth_target']
    stock_df = stock_df[columns_keep]

if domain_attributes and time_series_attributes:
    stock_df = stock_df
stock_df

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,...,3,8,3,26,34,16.82441,-17.72197,78.0,242.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.94, 3.71, -0...."
1,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,...,1,1,1,2,5,18.59324,-8.00000,54.0,58.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 3.45, -2.0, -2...."
2,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,...,2,9,3,28,37,11.23512,-6.57895,37.0,19.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -1.34, -1.36,..."
3,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,71.777,...,4,4,2,28,15,9.97164,-10.13175,18.0,220.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -1.34, 0.0, 0.0..."
4,Germany,Asset Management,EUR,Europe/Berlin,GER,Financial Services,11244.0,-2.942,3.008613e+08,37.175,...,0,5,2,29,19,14.24548,-26.26091,129.0,257.0,"[0.0, -2.56, -0.75, -0.38, 4.17, -1.64, 2.59, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7844,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,15.043,...,4,4,2,15,13,17.62885,-12.85629,50.0,219.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -1.43, 2.49, 1..."
7845,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,...,4,9,3,10,39,13.67960,-9.63956,245.0,54.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 2.97, 0.0, 0..."
7846,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,...,4,9,3,26,39,9.76675,-8.39559,167.0,41.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0, 2.3..."
7847,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,...,4,9,3,26,39,14.62789,-10.74116,55.0,115.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [6]:
window_size = [1,5,10,20,65]

In [7]:
for w_size in window_size:
    stock_df=stock_df_backup.copy()
    column_names = list(stock_df.columns)
    beam_width = 20
    beam_depth = 3
    nr_bins = 8
    nr_saved = 50
    subgroup_size = len(stock_df) * 0.05
    target = 'growth_target'
    target_ind = column_names.index(target)
    stock_df['growth_target'] = stock_df['growth_target'].apply(lambda x: x[:len(x) - (len(x) % w_size)])
    stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(lambda x: x[:len(x) - (len(x) % w_size)])
    data = stock_df.values.tolist()
    all_time_series = [i[target_ind] for i in data]
    all_time_series = np.array(all_time_series)
    targets_baseline = np.mean(all_time_series, axis=0)
    att_indices = list(range(0, len(column_names)))
    att_indices.remove(target_ind)
    att_columns = [column_names[i] for i in att_indices]
    types = bsm.categorize_columns_in_order(stock_df, att_columns)
    min_quality_improv = 0.1
    agg_func = np.mean
    results = bsm.beam_search_with_constraint(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved,
                          subgroup_size, target, types, w_size, min_quality_improv, agg_func)
    q_res = []
    for res in results:
        q_res.append(res[0])
    
    column_count = {}
    
    # Iterating through each tuple in the data list
    for item in results:
        conditions = item[1]  # The second element contains the conditions (column names)
        for condition in conditions:
            column_name = condition[0]  # The first element in the condition tuple is the column name
            # Increment the count for this column name
            if column_name in column_count:
                column_count[column_name] += 1
            else:
                column_count[column_name] = 1
    column_count = dict(sorted(column_count.items(), key=lambda x: x[1], reverse=True))
    
    print('window size:',w_size,' quality mean:', np.mean(q_res),' quality max:', max(q_res), ' quality std:', np.std(q_res), ' column_count:', column_count)
    print('\n')

window size: 1  quality mean: 0.10618175892768562  quality max: 0.17767278684959623  quality std: 0.02086945678110939  column_count: {'averageVolume10days': 12, 'growth_median': 7, 'growth_std_dev': 5, 'exchangeTimezoneName': 5, 'growth_biggest_continuous_increase_perc': 4, 'growth_range': 4, 'country': 4, 'exchange': 3, 'growth_biggest_continuous_decrease_perc': 3, 'growth_max': 2, 'growth_min': 2, 'currency': 2, 'marketCap': 2, 'fullTimeEmployees': 1, 'sector': 1, 'og_autocorr_lag1': 1, 'og_longest_continuous_decrease': 1, 'og_longest_continuous_increase': 1, 'growth_autocorr_lag1': 1, 'og_min': 1, 'debtToEquity': 1, 'og_mean': 1}


window size: 5  quality mean: 0.1412892911424718  quality max: 0.24845575106302123  quality std: 0.04202227204337382  column_count: {'growth_std_dev': 15, 'growth_min': 7, 'growth_biggest_continuous_increase_perc': 6, 'growth_mean': 6, 'growth_biggest_continuous_decrease_perc': 5, 'growth_range': 5, 'averageVolume10days': 5, 'growth_max': 3, 'og_window_mi